In [2]:
from google.colab import drive  
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
import pandas as pd
import math

In [6]:
PATH='/content/gdrive/MyDrive/개인 공부/dacon/dacon_energy/dacon_energy'

In [7]:
train = pd.read_csv(PATH + '/train.csv', encoding='cp949', header=0)
mis_test = pd.read_csv(PATH + '/test.csv', encoding='cp949', header=0)
sub = pd.read_csv(PATH + '/sample_submission.csv', encoding='cp949')

In [8]:
train.columns = ['num','datetime','power(kWh)','tempe(°C)','wind(m/s)','hum(%)','rain(mm)','sol(hr)','ne_cool','sol_energy']
mis_test.columns = ['num','datetime','tempe(°C)','wind(m/s)','hum(%)','rain(mm)','sol(hr)','ne_cool','sol_energy']

In [9]:
# 불쾌지수 계산 (T : 기온, H : 습도)
def discomfort(data, T, H):
  data['discomfort'] = 1.8 * T - 0.55 * (1-H) * (1.8 * T - 26) + 32

# 체감온도 계산을 위해서 풍속에 0.15 제곱이 필요
def get_pow(series):
    return math.pow(series, 0.15)

# 체감온도 계산 (T : 기온, V : 풍속)
def sensible(data, T, V):
  V = V.apply(get_pow)
  data['sensible'] = 13.12 + 0.6215 * T - 11.37 * V + 0.3965 * V * T

In [13]:
discomfort(train,train['tempe(°C)'] , train['hum(%)'] )
sensible(train,train['tempe(°C)'] , train['wind(m/s)'] )

In [14]:
train

,num,datetime,power(kWh),tempe(°C),wind(m/s),hum(%),rain(mm),sol(hr),ne_cool,sol_energy,discomfort,sensible
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,347.964,19.019755
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,353.930,19.014988
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,335.750,18.720323
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,299.390,18.282891
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,292.830,18.148033
...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0,967.914,30.004183
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0,972.030,29.526274
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0,972.030,29.491116
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0,995.397,29.280207


In [15]:
discomfort(mis_test,mis_test['tempe(°C)'] , mis_test['hum(%)'] )
sensible(mis_test,mis_test['tempe(°C)'] , mis_test['wind(m/s)'] )

In [17]:
mis_test.head()

,num,datetime,tempe(°C),wind(m/s),hum(%),rain(mm),sol(hr),ne_cool,sol_energy,discomfort,sensible
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN,1047.246,30.028622
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN,1061.119,29.533545
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
